In [15]:
!pip install transformers
!pip install tensorflow

import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from sklearn.metrics import confusion_matrix

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

df = pd.read_csv('edos_labelled_aggregated.csv')
df = df.drop(columns=["rewire_id", "label_category", "label_vector", "split"])
df = df.rename(columns={"label_sexist": "label"})
df['label'] = df['label'].map({'not sexist': 0, 'sexist': 1})


print(df.shape)
print("not sexist: ", len(df[df["label"] == 0]))
print("sexist: ", len(df[df["label"] == 1]))

(20000, 2)
not sexist:  15146
sexist:  4854


In [10]:
train_size = 0.8
train_data = df.sample(frac=train_size, random_state=200).reset_index(drop=True)
test_data = df.drop(train_data.index).reset_index(drop=True)
print(train_data. shape, test_data.shape)
print("TRAIN SPLIT:")
print("not sexist: ", len(train_data[train_data["label"] == 0]))
print("sexist: ", len(train_data[train_data["label"] == 1]))
print("TEST SPLIT:")
print("not sexist: ", len(test_data[test_data["label"] == 0]))
print("sexist: ", len(test_data[test_data["label"] == 1]))

(16000, 2) (4000, 2)
TRAIN SPLIT:
not sexist:  12113
sexist:  3887
TEST SPLIT:
not sexist:  3141
sexist:  859


In [11]:
train_encodings = tokenizer(train_data["text"].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_data["text"].tolist(), truncation=True, padding=True)

y_train = tf.convert_to_tensor(train_data["label"].values)
y_test = tf.convert_to_tensor(test_data["label"].values)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

In [12]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_dataset, epochs=2, validation_data=test_dataset)

results = model.evaluate(test_dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

y_pred = tf.argmax(model.predict(test_dataset).logits, axis=1).numpy()
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

model.save_pretrained('model')


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/2
2000/2000 [==============================] - 227s 101ms/step - loss: 0.3656 - accuracy: 0.8430 - val_loss: 0.2394 - val_accuracy: 0.9062
Epoch 2/2
250/250 [==============================] - 15s 60ms/step - loss: 0.1684 - accuracy: 0.9385
Test Loss: 0.16843774914741516, Test Accuracy: 0.9384999871253967
250/250 [==============================] - 18s 58ms/step
Confusion Matrix:
 [[3002  139]
 [ 107  752]]


In [16]:
y_test_array = y_test.numpy()

false_positives = np.where((y_pred == 1) & (y_test_array == 0))[0]
false_negatives = np.where((y_pred == 0) & (y_test_array == 1))[0]
true_positives = np.where((y_pred == 1) & (y_test_array == 1))[0]
true_negatives = np.where((y_pred == 0) & (y_test_array == 0))[0]

if false_positives.size > 0:
    print("False Positive Example:")
    print(test_data.iloc[false_positives[0]]['text'], "\n")

if false_negatives.size > 0:
    print("False Negative Example:")
    print(test_data.iloc[false_negatives[0]]['text'], "\n")

if true_positives.size > 0:
    print("True Positive Example:")
    print(test_data.iloc[true_positives[0]]['text'], "\n")

if true_negatives.size > 0:
    print("True Negative Example:")
    print(test_data.iloc[true_negatives[0]]['text'], "\n")

False Positive Example:
experince does count. however, older women generally want to settle down and look for status and other things. Id fuck 18-25 girls as long as i can 

False Negative Example:
^^^ This what it has come to today... THIS IS NORMAL! You can't trust woman at all today... and they STILL wonder why all the good men have gone MGTOW! 

True Positive Example:
A man's biggest mistake is trusting his girlfriend/wife to stay loyal. 

True Negative Example:
Really not true, you can say most women that can't get a guy to stay committed that they really want lack girl game, and lots lack girl game. 



In [17]:
# model 2
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
base_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

base_model.trainable = False

input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

base_output = base_model(input_ids, attention_mask=attention_mask)
last_hidden_state = base_output.last_hidden_state

x = tf.keras.layers.GlobalAveragePooling1D()(last_hidden_state)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)  # Layer dropout per regolarizzazione
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # Sigmoid per output binario

model2 = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model2.fit(train_dataset, epochs=10, validation_data=test_dataset)

results = model2.evaluate(test_dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

y_pred = model2.predict(test_dataset).round().astype(int)  # Ottieni predizioni binarie (0 o 1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

model2.save('model2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/10
2000/2000 [==============================] - 105s 47ms/step - loss: 0.5194 - accuracy: 0.7606 - val_loss: 0.4630 - val_accuracy: 0.7970
Epoch 2/10
2000/2000 [==============================] - 93s 46ms/step - loss: 0.4737 - accuracy: 0.7791 - val_loss: 0.4396 - val_accuracy: 0.8055
Epoch 3/10
2000/2000 [==============================] - 81s 40ms/step - loss: 0.4580 - accuracy: 0.7885 - val_loss: 0.4306 - val_accuracy: 0.8055
Epoch 4/10
2000/2000 [==============================] - 81s 41ms/step - loss: 0.4490 - accuracy: 0.7937 - val_loss: 0.4235 - val_accuracy: 0.8085
Epoch 5/10
2000/2000 [==============================] - 79s 40ms/step - loss: 0.4441 - accuracy: 0.7952 - val_loss: 0.4190 - val_accuracy: 0.8127
Epoch 6/10
2000/2000 [==============================] - 87s 43ms/step - loss: 0.4401 - accuracy: 0.7978 - val_loss: 0.4159 - val_accuracy: 0.8123
Epoch 7/10
2000/2000 [==============================] - 87s 44ms/step - loss: 0.4379 - accuracy: 0.7989 - val_loss: 0.4137 